In [1]:
import pandas as pd
import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm
import celloracle as co
from celloracle import motif_analysis as ma
from celloracle.utility import save_as_pickled_object
co.__version__

which: no R in (/home/gaojie/bin:/home/gaojie/.local/bin:/home/gaojie/bin:/opt/anaconda/envs/celloracle_env/bin:/opt/anaconda/condabin:/usr/share/Modules/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/gaojie/workspace/miniforge3/envs/glue/bin/bedtools:/home/gaojie/workspace/miniforge3/envs/glue/bin/bedtools)


'0.18.0'

In [2]:
os.chdir('/home/gaojie/workspace/Mida_collab/')

In [3]:
import genomepy

In [4]:
genomepy.install_genome(name="hg38", provider="UCSC")

Fasta("/home/gaojie/.local/share/genomes/hg38/hg38.fa")

In [5]:
ref_genome = "hg38"

genome_installation = ma.is_genome_installed(ref_genome=ref_genome,
                                             genomes_dir=None)
print(ref_genome, "installation: ", genome_installation)

hg38 installation:  True


In [6]:
gene2peak = pd.read_csv('Multi-omics_intergration/output/gene2peak_df_all.csv',index_col=0,header=0)

In [7]:
gene2peak['peak_id'] = [i.replace(':','_').replace('-','_') for i in gene2peak['peak_id']]

In [8]:
gene2peak.head()

,peak_id,gene_short_name,qval,score,pval
0,chr17_7662026_7662526,TP53,0.006082,0.873452,0.001170
1,chr17_7665809_7666309,TP53,0.006082,0.897910,0.000808
2,chr17_7666985_7667485,TP53,0.006082,0.900482,0.000757
3,chr17_7668423_7668923,TP53,0.006082,0.887318,0.000998
4,chr17_7669021_7669521,TP53,0.006082,0.873999,0.001152


In [9]:
gene2peak = ma.check_peak_format(gene2peak, ref_genome, genomes_dir=None)

Peaks before filtering:  113695
Peaks with invalid chr_name:  0
Peaks with invalid length:  0
Peaks after filtering:  113695


In [10]:
tfi = ma.TFinfo(peak_data_frame=gene2peak,
                ref_genome=ref_genome,
                genomes_dir=None)

In [11]:
tfi.scan(fpr=0.02,
         motifs=None,  # If you enter None, default motifs will be loaded.
         verbose=True,n_cpus=3)

No motif data entered. Loading default motifs for your species ...
 Default motif for vertebrate: gimme.vertebrate.v5.0. 
 For more information, please see https://gimmemotifs.readthedocs.io/en/master/overview.html 

Initiating scanner... 



2025-09-24 23:22:08,115 - DEBUG - using background: genome hg38 with size 200


Calculating FPR-based threshold. This step may take substantial time when you load a new ref-genome. It will be done quicker on the second time. 

Motif scan started .. It may take long time.



Scanning:   0%|          | 0/108954 [00:00<?, ? sequences/s]

In [12]:
tfi.scanned_df.head()

,seqname,motif_id,factors_direct,factors_indirect,score,pos,strand
0,chr10_100009027_100009527,GM.5.0.bHLH.0002,"TCF3, Tcfap4","ATOH7, Atoh7",9.057189,355,-1
1,chr10_100009027_100009527,GM.5.0.C2H2_ZF.0004,"ZIC1, Zic1","Zic4, ZIC5",6.875149,218,-1
2,chr10_100009027_100009527,GM.5.0.bHLH.0006,"Bhlha15, Neurog2, Olig2, PTF1A, Neurod1, NEURO...","GATA1, GATA2",9.248468,356,-1
3,chr10_100009027_100009527,GM.5.0.bHLH.0010,"NEUROG2, Msgn1","Bhlhe23, Olig2, Neurog2, NEUROD6, NEUROG3, NEU...",8.553253,355,1
4,chr10_100009027_100009527,GM.5.0.Homeodomain.0033,NKX2-5,"NKX2-6, NKX2-4, NKX2-3",9.901231,130,-1


In [13]:
tfi.to_hdf5(file_path="CellOracle/input/GW.11.celloracle.tfinfo")

In [14]:
tfi.reset_filtering()
# Do filtering
tfi.filter_motifs_by_score(threshold=10)
# Format post-filtering results.
tfi.make_TFinfo_dataframe_and_dictionary(verbose=True)

Filtering finished: 15867671 -> 3419896
1. Converting scanned results into one-hot encoded dataframe.


  0%|          | 0/108954 [00:00<?, ?it/s]

2. Converting results into dictionaries.


  0%|          | 0/5168 [00:00<?, ?it/s]

  0%|          | 0/1095 [00:00<?, ?it/s]

In [15]:
df = tfi.to_dataframe()
df.head()

,peak_id,gene_short_name,qval,score,pval,9430076C15RIK,AC002126.6,AC012531.1,AC226150.2,AFP,...,ZNF784,ZNF8,ZNF816,ZNF85,ZSCAN10,ZSCAN16,ZSCAN22,ZSCAN26,ZSCAN31,ZSCAN4
0,chr10_100009027_100009527,DNMBP,0.006143,0.806778,0.002391,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,chr10_100009744_100010244,DNMBP,0.011732,0.681134,0.009856,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,chr10_100910568_100911068,SLF2,0.006082,0.931589,0.000568,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,chr10_100912022_100912522,SLF2,0.006082,0.925895,0.000636,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,chr10_100912609_100913109,SLF2,0.014486,0.655965,0.012746,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df.to_parquet("CellOracle/input/base_GRN_dataframe.parquet")